In [ ]:
# ! pip install PyMuPDF

# # مكتبات عامة مهمة للـ pipelines
# !pip install pydantic tiktoken

# # لو هتشتغل مع HF GPTQ models
# !pip install accelerate bitsandbytes safetensors

# # للتعامل مع أي نوع من الـ LLMs أو embeddings
# !pip install langchain sentence-transformers
# !pip install faiss-cpu

# # Re-install/Update all relevant libraries to ensure compatibility
# !pip install -U transformers accelerate auto-gptq torch optimum


## ingestion/loader

In [1]:
# from zenml import step
import os
from typing import  List , Dict
from pathlib import Path
import hashlib
import logging

import fitz

'''
document-level parsing.
الهدف

نطلع وحدات نص نظيفة + metadata مضبوط
بحيث أي chunk بعد كده يعرف هو جاي منين بالظبط.

هنطلع Pages / Blocks بالشكل ده:
{
  "text": "...",
  "metadata": {
      "source": "doc_name",
      "page": 3,
      "language": "ar",
      "doc_type": "article"
  }
}

'''


# we will work on en + ar only so  we have to detect the lang

def detect_lang(text : str ) -> str :

    ar_chars = sum(1 for char in text if '\u0600' <= char <= '\u06FF')
    return "ar" if ar_chars > 10 else "en"

class DocumentLoader:
    def __init__(self , data_path : Path):
        self.path = Path(data_path)

    def _load_txt(self , data_path : Path) -> List[Dict]:
        '''' if text is a type of txt we use this method to extract it'''
        try :
            logging.info("extracting the text from txt file . ")
            text = data_path.read_text(encoding="utf-8")
            return [
                {
                    "text" : text ,
                    "metadata" : {
                        "source" : data_path.name ,
                        "page" : 1 ,
                        "language" : detect_lang(text) ,
                        "doc_type": "txt"
                    }
                }
            ]

        except Exception as e :
            logging.error("Couldin't extract the text .....")
            raise e


    def _load_pdf(self , data_path : Path) -> List[Dict]:
        '''' if text is a type of pdf we use this method to extract it'''

        pages = []
        try :
            logging.info(f"extracting the text from {data_path.name}.pdf file . ")
            doc = fitz.open(data_path)

            for page_number , page in enumerate(doc):
                text = page.get_text().strip()
                if not text :
                    continue


                pages.append({
                    "text": text,
                    "metadata": {
                        "source": data_path.name,
                        "page": page_number + 1,
                        "language": detect_lang(text),
                        "doc_type": "pdf"
                    }
                })
                logging.info(f"text extracted sussfully from {data_path.name}.pdf file ")
        except Exception as e :
            logging.error(f"Couldin't open and extracting the {data_path.name}.pdf file .....")
            raise e


        return pages



# @step(enable_cache=True)
def load(data_path : Path) -> List[Dict]:
        ''' a general method of extracting documents '''
        data_path = Path(data_path)
        loader = DocumentLoader(data_path)
        docs = []

        for file in data_path.iterdir():
            if file.suffix.lower() == '.txt' :
                docs.extend(loader._load_txt(file))
            elif file.suffix.lower() == ".pdf":
                docs.extend(loader._load_pdf(file))

        if len(docs) > 1 :
            logging.info("All Text Has been Extracted Sussfully .....")
        return docs



## Chunking Engine (Recursive)

In [3]:
from typing import List, Dict
import hashlib
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

class RecursiveChunker:
    def __init__(
        self,
        chunk_size: int = 300,
        overlap: int = 80
    ):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def chunk(self, documents: List[Dict]) -> List[Dict]:
        chunks = []

        for doc in documents:
            tokens = tokenizer.encode(doc["text"])
            start = 0

            while start < len(tokens):
                end = start + self.chunk_size
                chunk_tokens = tokens[start:end]
                chunk_text = tokenizer.decode(chunk_tokens)

                chunk_id = self._make_chunk_id(
                    chunk_text,
                    doc["metadata"]
                )

                chunks.append({
                    "id": chunk_id,
                    "text": chunk_text,
                    "metadata": doc["metadata"]
                })

                start += self.chunk_size - self.overlap

        return chunks

    def _make_chunk_id(self, text: str, metadata: Dict) -> str:
        base = (
            text
            + metadata["source"]
            + str(metadata["page"])
        )
        return hashlib.sha256(base.encode()).hexdigest()


# @step(enable_cache=True)
def chunk_documents(
    documents: List[Dict],
    chunk_size: int = 300,
    overlap: int = 80
) -> List[Dict]:
    chunker = RecursiveChunker()
    return chunker.chunk(documents)


## Chunks Embedding

In [4]:
'''
الهدف الحقيقي

تحويل كل chunk إلى vector:

normalized
reusable
cached
deterministic

بحيث:

❌ مفيش chunk يتعمله embedding مرتين
❌ مفيش drift
✅ أي إعادة ingestion تبقى cheap

المشكلة

لو عندك:
100k chunks

كل مرة تشغل السيستم بتعمل embedding من الأول

يبقى:
cost - latency - waste


✅ الحل

نعمل Embedding Cache Layer
يعتمد على:

🔑 Key => hash(chunk_text)


ليه مش chunk_id؟
chunk_id فيه metadata

embedding لازم يعتمد على النص فقط
'''
import os
import hashlib
from typing import List, Dict
import logging

# from zenml import step
import numpy as np
from sentence_transformers import SentenceTransformer

# إعداد logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

class EmbeddingCache:
    def __init__(self, cache_dir: str = "data/processed/embeddings"):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _path(self, text_hash: str) -> str:
        return os.path.join(self.cache_dir, f"{text_hash}.npy")

    def exists(self, text_hash: str) -> bool:
        return os.path.exists(self._path(text_hash))

    def load(self, text_hash: str) -> np.ndarray:
        return np.load(self._path(text_hash))

    def save(self, text_hash: str, vector: np.ndarray):
        np.save(self._path(text_hash), vector)


class Embedder:
    def __init__(self, model_name="BAAI/bge-m3", device="cuda", batch_size=32):
        self.model = SentenceTransformer(model_name, device=device)
        self.batch_size = batch_size
        self.cache = EmbeddingCache()

    def _hash(self, text: str) -> str:
        return hashlib.sha256(text.encode("utf-8")).hexdigest()

    def chunk_embeded(self, chunks):
        embedded = []
        texts_to_encode = []
        chunk_indices = []

        for i, chunk in enumerate(chunks):
            text = chunk["text"]
            text_hash = self._hash(text)
            if self.cache.exists(text_hash):
                vector = self.cache.load(text_hash)
                embedded.append({
                    "chunk_id": chunk["id"],
                    "vector": vector,
                    "metadata": chunk["metadata"]
                })
            else:
                texts_to_encode.append(text)
                chunk_indices.append(i)
                embedded.append(None)  # placeholder

        if texts_to_encode:
            logging.info(f"Encoding {len(texts_to_encode)} chunks in batches of {self.batch_size}...")
            vectors = self.model.encode(texts_to_encode, batch_size=self.batch_size, normalize_embeddings=True)
            for idx, vector in zip(chunk_indices, vectors):
                chunk = chunks[idx]
                text_hash = self._hash(chunk["text"])
                self.cache.save(text_hash, vector)
                embedded[idx] = {
                    "chunk_id": chunk["id"],
                    "vector": vector,
                    "metadata": chunk["metadata"]
                }

        logging.info("All chunks embedded ✅")
        x = embedded
        return x


# @step(enable_cache=True)
def chunks_embedding(chunks) :
    em = Embedder()
    return em.chunk_embeded(chunks)


## tst pipline

In [5]:
    documents = load(data_path="/content/data")
    chunks = chunk_documents(documents = documents ,chunk_size=300 , overlap = 80)
    embedeed_chunks = chunks_embedding(chunks)

In [6]:
embedeed_chunks


[{'chunk_id': '2137a9f0c3f18a9af5dd900194085f16a6e07590a5e483ca155c8a9291289a58',
  'vector': array([-0.0364996 , -0.00801616, -0.05606956, ..., -0.00564637,
         -0.0425735 ,  0.03824963], dtype=float32),
  'metadata': {'source': 'google.pdf',
   'page': 1,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': 'b7b13257da9ce3d9f82801926c405ba5c3600e59cedf1b981451234b6b91956f',
  'vector': array([-0.0683113 , -0.05868272, -0.03021523, ...,  0.01281825,
          0.01000643,  0.01353191], dtype=float32),
  'metadata': {'source': 'google.pdf',
   'page': 1,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': '23648fdd78f5f54dcdce63e3dc71650c2a12ae0bb0d4b519c4c22b7cff8d01a6',
  'vector': array([-0.03948649, -0.02500342, -0.02339096, ...,  0.01711106,
         -0.04269707,  0.00018443], dtype=float32),
  'metadata': {'source': 'google.pdf',
   'page': 1,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': 'a54d624941b3d51e28003f063332a2bc12910b2c2338b65eb7caa5e542be2

## faiss_index

In [7]:
'''
Docstring for steps.faiss_index
الهدف

نخزن embeddings في FAISS

نقدر نعمل Top-K retrieval بسرعة
نحافظ على ربط vector ↔ metadata / chunk_id
نقدر save / load index لأي وقت
'''
import faiss
import numpy as np
import os
import json
from typing import List, Dict

# from zenml import step

class FaissIndex:
    def __init__(self, vector_dim: int,
                  index_path: str = "data/processed/faiss.index",
                  mapping_path: str = "data/processed/faiss_mapping.json"):
        self.vector_dim = vector_dim
        self.index_path = index_path
        self.mapping_path = mapping_path

        self.index = faiss.IndexFlatIP(vector_dim)
        self.mapping = {}  # faiss_id -> chunk_id
        self.faiss_id = 0

    def add(self , embeddings: List[Dict]):
        for emb in embeddings :
            vector = emb['vector'].reshape(1, -1)
            self.index.add(vector)

            # Store chunk_id instead of metadata
            self.mapping[self.faiss_id] = emb['chunk_id'] # faiss_id => chunk_id in serach we will use the faiss_id that retures from the indexes and bring back the chunk_id and we will do a chunk_id mapping to metadata
            self.faiss_id +=1

    def search(self , query_vector: np.ndarray, top_k: int = 50) -> List[str]:
        query_vector = query_vector.reshape(1, -1)
        d , I = self.index.search(query_vector , top_k )
        # Return a list of chunk_ids
        return [self.mapping[i] for i in I[0]]

    def save(self):
        faiss.write_index(self.index, self.index_path)
        with open(self.mapping_path, "w", encoding="utf-8") as f:
            json.dump(self.mapping, f, ensure_ascii=False, indent=2)

    def load(self):
        if os.path.exists(self.index_path):
            self.index = faiss.read_index(self.index_path)
        if os.path.exists(self.mapping_path):
            with open(self.mapping_path, "r", encoding="utf-8") as f:
                # Keys loaded from JSON will be strings, convert them back to int
                self.mapping = {int(k): v for k, v in json.load(f).items()}
            self.faiss_id = max(self.mapping.keys()) + 1 if self.mapping else 0


# @step()
def faiss_index_step(embeddings: List[Dict], vector_dim: int):
    faiss_index = FaissIndex(vector_dim)
    faiss_index.add(embeddings)
    faiss_index.save()
    return faiss_index

In [8]:
faiss_index = faiss_index_step(embedeed_chunks , 1024)
faiss_index

## Prepare Metadata Store

In [9]:
chunk_metadata_store = {
    chunk["chunk_id"]: chunk["metadata"]
    for chunk in embedeed_chunks
}


## Retrieval + Filtering

In [10]:
class Retriever:
    def __init__(self, faiss_index, chunks_metadata: Dict[str, Dict]):
        """
        faiss_index: instance of FaissIndex
        chunks_metadata: dict chunk_id -> metadata
        """
        self.faiss_index = faiss_index
        self.chunks_metadata = chunks_metadata

    def query_retreviel(self,
                        query_vector: np.ndarray,
                        top_k: int = 50,
                        filters: Dict = None
                    ) -> List[Dict]:
      canditae_ids = self.faiss_index.search(query_vector , top_k * 2)

      results = []

      for cid in canditae_ids :
        meta = self.chunks_metadata.get(cid)

        if not meta :
          continue

        if "language" in filters and filters['language'] != meta['language'] :
          continue

        results.append({
                "chunk_id": cid,
                "metadata": meta
            })

      return results[:top_k]



## Testing retrevial and app so far

In [11]:
# Suppose we have a query vector
# query_vec = np.random.rand(1024).astype("float32")
model = SentenceTransformer("BAAI/bge-m3", device="cuda")
questions = "What was the direct financial impact of this change on the Income Statement for the year ended December 31, 2023? Be specific about the amounts for depreciation expense and net income."

query_vec = model.encode(questions, normalize_embeddings=True)

retriever = Retriever(faiss_index , chunk_metadata_store)

results = retriever.query_retreviel(
    query_vec,
    50,
    {"language": "en"}
)

In [ ]:
results[:5]

[{'chunk_id': 'e970a0b5f0faecefd8a25c6bae7e0a701e9a3037c5589cac110018bdb785c63a',
  'metadata': {'source': 'google.pdf',
   'page': 71,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': '37bff9113c872072c57678756d74956cd29c7a41a4716052532e3060ba15915f',
  'metadata': {'source': 'google.pdf',
   'page': 41,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': '7a4cc26970a08164528fd1bf504940725cc7462807fd5fd0c36c63cfbd70a35d',
  'metadata': {'source': 'google.pdf',
   'page': 85,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': 'b8bbd8a273b54f804d19d3e3d9227fc01807fa3875e156ba174ea44f356d229c',
  'metadata': {'source': 'google.pdf',
   'page': 76,
   'language': 'en',
   'doc_type': 'pdf'}},
 {'chunk_id': '2a4e92e9fe0a73c24cf12857ede8ec471db74a7fb4b659f5ae14a7477964caad',
  'metadata': {'source': 'google.pdf',
   'page': 41,
   'language': 'en',
   'doc_type': 'pdf'}}]

## Cross-Encoder Re-Ranking

In [25]:
# الهدف

# ناخد Top-50 chunks من Retriever
# نعيد ترتيبهم باستخدام Cross-Encoder
# نطلع Top-5 فقط بأعلى دقة ممكنة

from sentence_transformers import CrossEncoder

class CrossEncoderReRanker:
    def __init__(self, model_name: str = "BAAI/bge-reranker-base" ,
                 device = "cuda" ,
                 top_k: int = 5) -> List[Dict]:

        self.model = CrossEncoder(model_name , device=device)

    def CE(self , query , rsults_from_faiss , top_k = 5):
        chunks_ids_from_faiss = [r['chunk_id'] for r in rsults_from_faiss]
        texts = [chunks[i]['text'] for i in range(len(chunks)) if chunks[i]['id'] in chunks_ids_from_faiss]


        pairs = [
            (query, t) for t in texts
        ]

        scores = self.model.predict(pairs)

        scored = []
        for c, score in zip(rsults_from_faiss, scores):
            scored.append({
                **c,
                "score": float(score)
            })

        scored.sort(key=lambda x: x["score"], reverse=True)

        return scored[:top_k]


In [26]:
reranker = CrossEncoderReRanker(
    model_name="BAAI/bge-reranker-base",
    device="cuda"
)

top_chunks = reranker.CE(
    questions,
    results
)


In [27]:
top_chunks

[{'chunk_id': 'f61637a5be9ccbbb0a7b7555dfb1690883777ce6aa5debcd0d499d98fb7c1842',
  'metadata': {'source': 'google.pdf',
   'page': 72,
   'language': 'en',
   'doc_type': 'pdf'},
  'score': 0.995226263999939},
 {'chunk_id': 'e56e30da087edd98c78d57c70532009c71a549e0bd43bae0901fd740ccda6fbd',
  'metadata': {'source': 'google.pdf',
   'page': 41,
   'language': 'en',
   'doc_type': 'pdf'},
  'score': 0.9753535389900208},
 {'chunk_id': '10b5c3278aead3ff9f4164bf9f0d88c9ff99c78c4a22ddd78bb41f1a637e80ec',
  'metadata': {'source': 'google.pdf',
   'page': 41,
   'language': 'en',
   'doc_type': 'pdf'},
  'score': 0.9271805882453918},
 {'chunk_id': '8f8c237c5d126e18cf364efff0e317db6c281c1f096d461a51f6423a7155386d',
  'metadata': {'source': 'google.pdf',
   'page': 40,
   'language': 'en',
   'doc_type': 'pdf'},
  'score': 0.9218613505363464},
 {'chunk_id': '5cfd49b5b24fa96afa072e027874f78766442fa699920ce9f28df9326911ecaf',
  'metadata': {'source': 'google.pdf',
   'page': 56,
   'language': 'e

In [33]:
def add_txt_to_top_chunks(top_chunks, chunks):
    # نمر على كل chunk في top_chunks
    for c in top_chunks:
        # نبحث إذا chunk_id موجود في chunks
        for t in chunks:
            if c['chunk_id'] == t['id']:
                # نضيف نص جديد للمفتاح 'text'
                c['text'] = t['text']
                break
    return top_chunks
top_chunks = add_txt_to_top_chunks(top_chunks, chunks)

In [38]:
top_chunks[0]

{'chunk_id': 'f61637a5be9ccbbb0a7b7555dfb1690883777ce6aa5debcd0d499d98fb7c1842',
 'metadata': {'source': 'google.pdf',
  'page': 72,
  'language': 'en',
  'doc_type': 'pdf'},
 'score': 0.995226263999939,
 'text': ' $ \n(28) $ \n319 \nAs of December 31, 2023\nGross Amounts Not Offset in \nthe Consolidated Balance \nSheets, but Have Legal Rights \nto Offset\nGross \nAmounts\nRecognized\nGross \nAmounts \nOffset in the \nConsolidated \nBalance \nSheets\nNet Amounts \nPresented in \nthe \nConsolidated \nBalance \nSheets\nFinancial \nInstruments(1)\nCash and \nNon-Cash \nCollateral \nReceived or \nPledged\nNet Amounts\nDerivatives assets\n$ \n535 $ \n(82) $ \n453 $ \n(213) $ \n(75) $ \n165 \nDerivatives liabilities\n$ \n527 $ \n(82) $ \n445 $ \n(213) $ \n(16) $ \n216 \n(1) \nThe balances as of December 31, 2022 and 2023 were related to derivatives allowed to be net settled in accordance with our \nmaster netting agreements.\nNote 4.        Leases \nWe have entered into operating lease agree

## Prompt Engineering + Anti-Hallucination

In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

class QwenLLMGenerator4bit:
    def __init__(self, model_name="Qwen/Qwen2.5-1.5B-Instruct"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # 4-bit quantization configuration
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        ).to(self.device)

        self.system_prompt = """
مهمتك هي الإجابة على السؤال باستخدام النص المعطى فقط.
❗ قواعد إلزامية:
- استخدم فقط المعلومات الموجودة في السياق.
- ممنوع التخمين أو الاستنتاج.
- لو الإجابة غير موجودة بوضوح، قل حرفيًا: "لا أعلم".
- أجب باللغة العربية وبإجابة مباشرة.
"""

    def build_prompt(self, question: str, chunks: list):
        context_blocks = []
        sources = []

        for i, c in enumerate(chunks, start=1):
            source = f"{c['metadata']['source']}:page{c['metadata'].get('page', '')}"
            sources.append(source)
            context_blocks.append(f"[{i}] المصدر: {source}\n{c['text']}")

        context = "\n\n".join(context_blocks)
        prompt = f"""
السياق:
{context}

السؤال:
{question}

الإجابة:
"""
        return prompt, sources

    def generate(self, prompt: str):
        full_prompt = f"### System:\n{self.system_prompt}\n\n### User:\n{prompt}\n\n### Assistant:\n"
        inputs = self.tokenizer(full_prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=128,
                # temperature=0.1,
                # top_p=0.9,
                # do_sample=False
            )

        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return decoded.split("### Assistant:")[-1].strip()


In [37]:
generator = QwenLLMGenerator4bit()

prompt, sources = generator.build_prompt(questions, top_chunks)
answer = generator.generate(prompt)

print(answer)
print("المصادر:", sources)

The change had no direct financial impact on the Income Statement for the year ended December 31, 2023. Specifically, there is no mention of any adjustments or changes that would affect the reported numbers directly. The only notable change mentioned is the decrease in the effective tax rate from 2022 to 2023, which does not relate to the financial statement amounts provided. Therefore, the answer should focus on providing the relevant details without making assumptions about the impact on the financial statements. 

If you need a more detailed response or if I misunderstood something, please let me know! 😊

### Answer
المصادر: ['google.pdf:page72', 'google.pdf:page41', 'google.pdf:page41', 'google.pdf:page40', 'google.pdf:page56']
